<a href="https://colab.research.google.com/github/TeamMAMI/MAMI/blob/overlaid_text_removal/overlaid_text_removal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://pasaentuciudad.com.mx/remove-text-from-images-using-cv2-and-keras-ocr/

Don't run the last code block until you've checked which directory you're in

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install keras-ocr

In [3]:
import cv2
import math
import glob
import keras_ocr
import os, os.path
import numpy as np
import pandas as pd
from pandas import read_csv
from tqdm import tqdm
from PIL import Image as pil_image
from numpy import asarray
import matplotlib.pyplot as plt
from IPython.display import Image as ip_image

Define the midpoint and inpaint text functions

In [4]:
def midpoint(x1, y1, x2, y2):
    x_mid = int((x1 + x2)/2)
    y_mid = int((y1 + y2)/2)
    return (x_mid, y_mid)

In [5]:
 def inpaint_text(img_path, pipeline):
    # read image
    img = img_path

    # generate (word, box) tuples 
    prediction_groups = pipeline.recognize([img])
    mask = np.zeros(img.shape[:2], dtype="uint8")
    for box in tqdm(prediction_groups[0]):
        x0, y0 = box[1][0]
        x1, y1 = box[1][1] 
        x2, y2 = box[1][2]
        x3, y3 = box[1][3] 
        
        x_mid0, y_mid0 = midpoint(x1, y1, x2, y2)
        x_mid1, y_mi1 = midpoint(x0, y0, x3, y3)
        
        thickness = int(math.sqrt( (x2 - x1)**2 + (y2 - y1)**2 ))
        
        cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mi1), 255,    
        thickness)
        img = cv2.inpaint(img, mask, 7, cv2.INPAINT_NS)

    return(img)

make the pipeline

In [6]:
pipeline = keras_ocr.pipeline.Pipeline()

Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5


make a list of all the image names in the trial set

In [7]:
# df = pd.DataFrame(data=read_csv('trial.csv', sep='\t'))

# trial_images = list(set(df["file_name"].tolist()))
# print(sorted(trial_images))
# print(len(trial_images))

...or do this for the training set

In [8]:
%cd drive/Shareddrives/team_MAMI/MAMI/TRAIN 
!pwd
!ls

/content/drive/Shareddrives/team_MAMI/MAMI/TRAIN
/content/drive/Shareddrives/team_MAMI/MAMI/TRAIN
Cleaned_Images	Images	training.csv


In [ ]:

df = pd.DataFrame(data=read_csv('training.csv', sep='\t'))

train_images = list(set(df["file_name"].tolist()))
print(sorted(train_images))
print(len(train_images))

In [10]:
# %mkdir Cleaned_Images

In [11]:
%cd drive/Shareddrives/team_MAMI/MAMI/TRIAL
!pwd
!ls

[Errno 2] No such file or directory: 'drive/Shareddrives/team_MAMI/MAMI/TRIAL'
/content/drive/Shareddrives/team_MAMI/MAMI/TRAIN
/content/drive/Shareddrives/team_MAMI/MAMI/TRAIN
Cleaned_Images	Images	training.csv


Pause here and make sure you're in the right directory

In [12]:
!pwd
!ls

/content/drive/Shareddrives/team_MAMI/MAMI/TRAIN
Cleaned_Images	Images	training.csv


Loop through the list of image names, opening, processing, and saving as pngs

In [13]:
# trial_images_path = "./Images/"
# for img in tqdm(trial_images):
  # image = asarray(cv2.imread(trial_images_path + img))
  # hold = inpaint_text(image, pipeline)
  # img_num = img.split('.')[0]
  # cv2.imwrite(f'/content/drive/Shareddrives/team_MAMI/MAMI/TRIAL/Clean_images/{img_num}.png',hold)

... or do this for the training images

but first get a list of the unprocessed images

In [14]:
%cd Cleaned_Images
!pwd

/content/drive/Shareddrives/team_MAMI/MAMI/TRAIN/Cleaned_Images
/content/drive/Shareddrives/team_MAMI/MAMI/TRAIN/Cleaned_Images


In [16]:
cleaned_images = os.listdir()
print(len(cleaned_images))

10000


In [ ]:
train_nums = [name.split('.')[0] for name in train_images]
print(train_nums)
print(len(train_nums))
print(len(train_images))
cleaned_nums = [name.split('.')[0] for name in cleaned_images]
print(cleaned_nums)
print(len(cleaned_nums))
print(len(cleaned_images))

In [18]:
unprocessed = list(set(train_nums) - set(cleaned_nums))

print(len(unprocessed))

0


In [ ]:
%cd ..
!pwd
!ls

In [ ]:
train_images_path = "./Images/"
for img in tqdm(unprocessed):
  image = asarray(cv2.imread(train_images_path + img + '.jpg'))
  hold = inpaint_text(image, pipeline)
  # img_num = img.split('.')[0]
  cv2.imwrite(f'/content/drive/Shareddrives/team_MAMI/MAMI/TRAIN/Cleaned_Images/{img}.png',hold)